In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', None)

df = pd.read_csv('analysts_survey_1.1.csv')

In [ ]:
df.columns
df.info()
#df #298 rows

In [6]:
#cleaning 

df.dropna(axis=0, how="any", thresh=None, subset=['avg_sallary'], inplace=True)
# 260 rows (38 rows didnt have salary)
df['avg_sallary'].isna().sum()

df = df[df['job_percentage'] == 'Full-time']
# 247 rows (13 rows didnt have salary)
df['job_percentage'].isna().sum()

df = df[df['data_analyzing'] == 'Yes']
# still 247 rows
df['data_analyzing'].isna().sum()

#clean people who work at the IDF - since their salary was low 
df.drop(df[df['company'] == 'IDF'].index, inplace=True)
# 239 rows (8 worked at the IDF)


### for the regression model:
# # drop the 1 na so i can use this field in the regression model
# df.dropna(axis=0, how="any", thresh=None, subset=['course_taken'], inplace=True)
# # 238 rows (1 dropped)
# df['course_taken'].isna().sum()

# df.dropna(axis=0, how="any", thresh=None, subset=['equity'], inplace=True)
# # 230 rows (8 nulls for equity)
# df['equity'].isna().sum()

In [7]:
#df #239 rows

In [9]:
#fired 
df[['fired_covid19']].describe()
# 19 people were fired

,fired_covid19
count,239
unique,2
top,No
freq,220


In [10]:
#wages_decrease_covid19
df[['wages_decrease_covid19']].describe()
# 27 wages were decreased

,wages_decrease_covid19
count,239
unique,2
top,No
freq,212


In [11]:
#preparing the data for the model - changing text values to numbers
df['gender'].replace(['Male', 'Female'], [0, 1], inplace=True)
df['data_science'].replace(['No', 'Yes'], [0, 1], inplace=True)
df['using_sql'].replace(['No', 'Yes'], [0, 1], inplace=True)
df['predictive_models'].replace(['No', 'Yes'], [0, 1], inplace=True)
df['wages_decrease_covid19'].replace(['No', 'Yes'], [0, 1], inplace=True)
df['fired_covid19'].replace(['No', 'Yes'], [0, 1], inplace=True)
df['equity'].replace(['No', 'Yes'], [0, 1], inplace=True)
df['manager'].replace(['No', 'Yes'], [0, 1], inplace=True)
df['course_taken'].replace(['No', 'Yes'], [0, 1], inplace=True)

#required_statistic_knowledge
df['required_statistic_knowledge'].replace(['No', 'Little', 'Medium', 'High'], [0, 1, 2, 3], inplace=True)

# cleaning but anyway - wont be in the model since all values are the same : Yes 
df['data_analyzing'].replace(['Yes'], [1], inplace=True)

In [12]:
#fired & wages decrease
df_fired_wages = df[["company", "fired_covid19", "wages_decrease_covid19", "startup_size"]]
df_fired_wages

,company,fired_covid19,wages_decrease_covid19,startup_size
0,Startup,1,0,Unkown
2,Corporate,0,0,NaN
3,Startup,0,0,Unkown
4,Startup,0,0,Small Startup (20-50)
5,Startup,0,0,Unkown
6,GAMFA,0,0,NaN
7,Startup,0,0,Unkown
8,GAMFA,0,0,NaN
9,Corporate,0,0,NaN
10,GAMFA,0,0,NaN


In [13]:
df_fired_wages["wages_decrease_covid19"].groupby(df_fired_wages["company"]).aggregate([np.count_nonzero])

,count_nonzero
company,
Corporate,10
Data consulting,0
GAMFA,3
Management consulting,0
Other,0
Public sector,0
Startup,14
Venture Capital,0


In [14]:
df_fired_wages["fired_covid19"].groupby(df_fired_wages["company"]).aggregate([np.count_nonzero])

,count_nonzero
company,
Corporate,10
Data consulting,0
GAMFA,4
Management consulting,0
Other,0
Public sector,0
Startup,5
Venture Capital,0


In [15]:
# check startup size - fired
df_fired_wages["fired_covid19"].groupby(df_fired_wages["startup_size"]).aggregate([np.count_nonzero])

,count_nonzero
startup_size,
Big Startup (300+),0
Early Stage Startup (0-20),0
Medium Startup (50-300),0
Small - medium,1
Small Startup (20-50),1
Unkown,3
Unkown,0


In [16]:
# check startup size - wage decrease
df_fired_wages["wages_decrease_covid19"].groupby(df_fired_wages["startup_size"]).aggregate([np.count_nonzero])

,count_nonzero
startup_size,
Big Startup (300+),0
Early Stage Startup (0-20),1
Medium Startup (50-300),0
Small - medium,1
Small Startup (20-50),5
Unkown,7
Unkown,0


In [17]:
#fired wages decrease
df.groupby('fired_covid19')[['wages_decrease_covid19']].mean().reset_index()

,fired_covid19,wages_decrease_covid19
0,0,0.090909
1,1,0.368421
